In [ ]:
%matplotlib inline
import scipy.special, cmath
import random, math, pylab, os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pylab
import mpl_toolkits.mplot3d
from matplotlib.patches import FancyArrowPatch, Circle, Rectangle

In [ ]:
#padding on the y axis between label and axis
plt.rcParams['ytick.major.pad']='10'

dpi_out = 400
fig_width = 3.39
golden_mean = (np.sqrt(5)-1.0)/2.0    # Aesthetic ratio
fig_height = fig_width*golden_mean # height in inches
MAX_HEIGHT_INCHES = 8.0
if fig_height > MAX_HEIGHT_INCHES:
    print("WARNING: fig_height too large:" + fig_height + 
          "so will reduce to" + MAX_HEIGHT_INCHES + "inches.")
    fig_height = MAX_HEIGHT_INCHES
#fig, ax = plt.subplots(figsize = (fig_width, fig_height), dpi=400, frameon=True)
small_tick_size = 8
small_label_size = 8

## Naive calculation of the the energy in the $2 \times 2$ Ising model 

In [ ]:
def energy(S, N, nbr):
    E = 0.0
    for k in range(N):
        E -=  S[k] * sum(S[nn] for nn in nbr[k])
    return 0.5 * E

L = 2
nbr = [[1, 2], [3, 0], [3, 0], [2, 1]]
S = [1, 1, -1, 1]
print S, energy(S, L * L, nbr)

## Gray code for Ising model

In [ ]:
def gray_flip(t):
    N = len(t) - 1
    k = t[0]
    if k > N: 
        return t, k
    t[k - 1] = t[k]
    t[k] = k + 1
    if k != 1: 
        t[0] = 1
    return k, t

### Enumeration of configurations

In [ ]:
def plot_spin(c, filename, L):
    s = 1.0 / L
    for i in range(L):
        for j in range(L):
            x, y, dy = (i + 0.5) * s, (j + 0.5) * s, 0.85 * s * c[i][j]
            arrow = FancyArrowPatch((x, y - 0.5 * dy), (x, y + 0.5 * dy),
                    color='.2', lw=0, alpha=.8, arrowstyle="Simple" +
                    ", head_length=" + str(1.3 * 150 * s) +
                    ", head_width=" + str(1.3 * 150 * s) +
                    ", tail_width=" + str(1.3 * 40 * s))
            pylab.gca().add_patch(arrow)
    pylab.axis('scaled')
    pylab.axis([0, 1, 0, 1])
    pylab.gca().set_xticks([])
    pylab.gca().set_yticks([])
    [pylab.axhline(y=(i * s), ls='--', c='.2') for i in range(L)]
    [pylab.axvline(x=(j * s), ls='--', c='.2') for j in range(L)]
    pylab.savefig(filename)
    pylab.show()
    pylab.clf()

In [ ]:
L = 2
N = L * L
site_dic = {(j // L, j - (j // L) * L) : j for j in range(N)}
print site_dic
S = [-1] * N
plot_spin([[S[site_dic[(a, b)]] for a in range(L)] for b in range(L)], 'spin_config_%04i.png' % 0, L)
tau = range(1, N + 2)
for i in range(1, 2 ** N):
    k, tau = gray_flip(tau)
    S[k - 1] *= -1
    plot_spin([[S[site_dic[(a, b)]] for a in range(L)] for b in range(L)], 'spin_config_%04i.png' % i, L)

### density of states for lattice with periodic boundary conditions

In [ ]:
L = 4
N = L * L
nbr = {i : ((i // L) * L + (i + 1) % L, (i + L) % N,
            (i // L) * L + (i - 1) % L, (i - L) % N)
                                    for i in range(N)}

S = [-1] * N
E = -2 * N
dos = {}
dos[E] = 1
tau = range(1, N + 2)
for i in xrange(1, 2 ** N):
    k, tau = gray_flip(tau)
    h = sum(S[n] for n in nbr[k - 1])
    E += 2 * h * S[k - 1] 
    S[k - 1] *= -1
    if E in dos: 
        dos[E] += 1
    else:        
        dos[E] = 1
for E in sorted(dos.keys()):
    print E, dos[E]

In [ ]:
fig, ax = plt.subplots(figsize = (3*fig_width, 3*fig_height), dpi=400, frameon=True)
ax.plot([E for E in sorted(dos.keys())], [dos[E] for E in sorted(dos.keys())])

## Thermodynamics: specific heat capacity

In [ ]:
L = 6
N = L * L
filename = 'data_dos_L%i.txt' % L

if os.path.isfile(filename):
    dos = {}
    f = open(filename, 'r')
    for line in f:
        E, N_E = line.split()
        dos[int(E)] = int(N_E)
    f.close()
else:
    print('input file missing')

In [ ]:
list_T = [0.5 + 0.01 * i for i in range(500)]
list_cv = []
for T in list_T:
    Z = 0.0
    E_av = 0.0
    M_av = 0.0
    E2_av = 0.0
    for E in dos.keys():
        weight = math.exp(- E / T) * dos[E]
        Z += weight
        E_av += weight * E
        E2_av += weight * E ** 2
    E2_av /= Z
    E_av /= Z
    cv = (E2_av - E_av ** 2) / N / T ** 2
    list_cv.append(cv)

In [ ]:
pylab.title('Specific heat capacity ($%i\\times%i$ lattice, PBC\'s)' % (L, L))
pylab.xlabel('$T$', fontsize=20)
pylab.ylabel('$c_V$', fontsize=20)
pylab.plot(list_T, list_cv, 'k-', lw=3)
pylab.show()

# Monte Carlo algorithms for the Ising model

## MCMC approach

# EXERCISE

Simulate the magnetisation curve as a function of the temperature for the Ising model on an $8 \times 8$ lattice.
Use a MCMC Metropolis algorithm using single spin flips: for each temperature in list_T = [1.0 + 0.2 * i for i in range(15)]:
1. Use nsteps = 10000 * N, with N = number of lattice sites.
2. let the system thermalize for nsteps / 2 steps.
3. take a snapshot of the magnetization every N steps after that.

The plotting facility is provided in the next cell

In [ ]:
## Implementation goes here

In [ ]:
pylab.title('$%i\\times%i$ lattice' % (L, L))
pylab.xlabel('$T$', fontsize=16)
pylab.ylabel('$<|M|>/N$', fontsize=16)
pylab.plot(list_T, list_av_m, 'bo-', clip_on=False)
pylab.ylim(0.0, 1.0)
pylab.show()
pylab.savefig('plot_local_av_magnetization_L%i.png' % L)

## Heatbath algorithm

In [ ]:
L = 6
N = L * L
nbr = {i : ((i // L) * L + (i + 1) % L, (i + L) % N,
            (i // L) * L + (i - 1) % L, (i - L) % N) for i in range(N)}
nsteps = 10000000
beta = 1.0
S = [random.choice([-1, 1]) for site in range(N)]
E = -0.5 * sum(S[k] * sum(S[nn] for nn in nbr[k]) for k in range(N))
E_tot, E2_tot = 0.0, 0.0
random.seed('123456')

for step in range(nsteps):
    k = random.randint(0, N - 1)
    Upsilon = random.uniform(0.0, 1.0)
    h = sum(S[nn] for nn in nbr[k])
    Sk_old = S[k]
    S[k] = -1
    if Upsilon < 1.0 / (1.0 + math.exp(-2.0 * beta * h)):
        S[k] = 1
    if S[k] != Sk_old:
        E -= 2.0 * h * S[k]
    E_tot += E
    E2_tot += E ** 2
E_av  = E_tot / float(nsteps)
E2_av = E2_tot / float(nsteps)
c_V = beta ** 2 * (E2_av - E_av ** 2) / float(N)

## Ising model: half order and perfect sampling

In [ ]:
output_dir = 'snapshots'
if not os.path.exists(output_dir): 
    os.makedirs(output_dir)
    
def plot_many_configurations(conf, filename, L, colors={}):
    pylab.figure(figsize=(6 * len(conf), 6))
    s = 1.0 / L
    for i_c in range(len(conf)):
        c = conf[i_c]
        pylab.subplot(1, len(conf), i_c + 1)
        for l in range(L ** 2):
            x, y = ((l // L) + 0.5) * s, ((l - (l // L) * L) + 0.5) * s
            dy = c[l] * 0.85 / float(L)
            arrow = FancyArrowPatch((x, y - 0.5 * dy), (x, y + 0.5 * dy), \
                    fc=colors[l], color='.2', lw=0, alpha=.8, \
                    arrowstyle="Simple, head_length=" + str(0.9 * 150 * s) \
                    + ", head_width=" + str(0.9 * 150 * s) + ", tail_width=" \
                    + str(0.9 * 40 * s))
            pylab.gca().add_patch(arrow)
        pylab.axis('scaled')
        pylab.axis([0, 1, 0, 1])
        pylab.gca().set_xticks([])
        pylab.gca().set_yticks([])
        [pylab.axhline(y=(i * s), ls='--', c='.2', lw=0.5) for i in range(L)]
        [pylab.axvline(x=(j * s), ls='--', c='.2', lw=0.5) for j in range(L)]
    pylab.tight_layout()
    pylab.savefig(output_dir + '/' + filename)
    pylab.show()
    pylab.clf()

In [ ]:
L = 6
N = L * L
nbr = {i : ((i // L) * L + (i + 1) % L, (i + L) % N,
            (i // L) * L + (i - 1) % L, (i - L) % N) \
                                    for i in range(N)}

filename = 'data_L%i.txt' % L
if os.path.isfile(filename):
    f = open(filename, 'r')
    S1 = [int(i) for i in f.read().split()]
    f.close()
    if len(S1) != N: exit('wrong input')
    print 'initial config read from', filename
else:
    S1 = [random.choice([-1, 1]) for i in range(N)]
    print 'random initial config'
    
S2 = [1] * N
nsteps = 10000
nskip  = 10     # plot a snapshot every nskip steps
beta = 0.4
random.seed('abcde')

for step in range(nsteps):
    k = random.randint(0, N - 1)
    Upsilon = random.uniform(0.0, 1.0)
    h1 = sum(S1[nn] for nn in nbr[k])
    S1[k] = -1
    if Upsilon < 1.0 / (1.0 + math.exp(-2.0 * beta * h1)): 
        S1[k] = 1
    h2 = sum(S2[nn] for nn in nbr[k])
    S2[k] = -1
    if Upsilon < 1.0 / (1.0 + math.exp(-2.0 * beta * h2)):
        S2[k] = 1
    if S1 == S2:
        print 'step %i: coupling' % step
        plot_many_configurations([S1, S2], 'snap_%06i.png' % step, L, {j : 'g' for j in range(N)})
        break
    else:
        print 'step %i: no coupling yet, %i different spins' % (step, sum(S1[ii] != S2[ii] for ii in range(N)))
    # begin graphic output
    # colormap: green for equal spins, red for different spins, blue for site k
    if step % nskip == 0:
        colors = {j : (S1[j] == S2[j]) * 'g' + (S1[j] != S2[j]) * 'r' for j in range(N)}
        colors[k] = 'b'
        plot_many_configurations([S1, S2], 'snap_%06i.png' % step, L, colors)

## Cluster Ising

Implement Wolff's algorithm for a $100 \times 100$ lattice
start for T = 2.5 (close to the phase transition)
execute nsteps = 10000 runs.
plot the configuration every delta_plot = 1000 runs (plot 10 different configs), starting from a random configuration - the plotting snippet below will do this efficiently.

In [ ]:


    if ((step % delta_plot) == 0):
        fig, ax = plt.subplots(figsize = (fig_width, fig_width), dpi=400, frameon=True)
        to_plot = np.resize(np.array(S), (L,L))
        plt.imshow(to_plot)
        ax.set_aspect('equal')